In [1]:
import numpy as np
from itertools import product
import pandas as pd
import os
import foraging_toolkit as ft
import environments
import simulation
import success_metrics
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s")


Parameters of the forward model. Specify the parameter values for all the cases you are interested in. 

In [2]:
# # Simulation setup 1 for the communication detection problem

home_dir = "../data/communicators/communicators_strong/"
# # agent parameters
sight_radius = [5]
c_trust = [
    0.0,
    0.5,
    0.7,
]  # 0: ignorers, 0.5: communicators (equal weight to own and others' knowledge of food)
N_agents = 9

# # environment parameters
edge_size = 30
N_total_food_units = 16
reward_patch_dim = [4]  # clustered is 4, distributed is 1

# simulation parameters
N_runs = 2  # How many times would you like to run each case?
N_frames = 150

# Generate a dataframe containing all possible combinations of the parameter values specified above.
param_list = [i for i in product(c_trust, sight_radius, reward_patch_dim)]
metadataDF = pd.DataFrame(param_list)
metadataDF.columns = ["c_trust", "sight_radius", "reward_patch_dim"]
metadataDF["sim index"] = np.arange(len(metadataDF)).astype(int)
N_sims = len(metadataDF)

# save metadata to home directory
metadataDF.to_csv(os.path.join(home_dir, "metadataDF.csv"))
pd.DataFrame(
    [
        {
            "N_sims": N_sims,
            "N_runs": N_runs,
            "N_frames": N_frames,
            "N_agents": N_agents,
            "N_total_food_units": N_total_food_units,
            "edge_size": edge_size,
        }
    ]
).to_csv(os.path.join(home_dir, "additional_meta_params.csv"))

display(metadataDF)

print(metadataDF.shape)

,c_trust,sight_radius,reward_patch_dim,sim index
0,0.0,5,4,0
1,0.5,5,4,1
2,0.7,5,4,2


(3, 4)


In [2]:
# custom list of locations for simulations
# with focus on low values of c_trust
min_value = 0.0
max_value = 0.7
density1 = 0.005
density2 = 0.01
c_locations = []

current_value = min_value
while current_value < 0.3:
    c_locations.append(current_value)
    current_value += density1
while current_value <= max_value:
    c_locations.append(current_value)
    current_value += density2

print(len(c_locations))


100


In [3]:
# Simulation setup 2 for the impact of communication

home_dir = "../data/communicators/communicators_data"

# agent parameters
sight_radius = [5]
c_trust = c_locations
# 0: ignorers, 0.5: mid-communicators (equal weight to own and others' knowledge of food)
N_agents = 9

# environment parameters
edge_size = 45
N_total_food_units = 16
reward_patch_dim = [1, 2, 4]  # clustered is 4, distributed is 1

# simulation parameters
N_runs = 2  # How many times would you like to run each case?
N_frames = 50

# Generate a dataframe containing all possible combinations of the parameter values specified above.
param_list = [i for i in product(c_trust, sight_radius, reward_patch_dim)]
metadataDF = pd.DataFrame(param_list)
metadataDF.columns = ["c_trust", "sight_radius", "reward_patch_dim"]
metadataDF["sim index"] = np.arange(len(metadataDF)).astype(int)
N_sims = len(metadataDF)

# save metadata to home directory
metadataDF.to_csv(os.path.join(home_dir, "metadataDF.csv"))
pd.DataFrame(
    [
        {
            "N_sims": N_sims,
            "N_runs": N_runs,
            "N_frames": N_frames,
            "N_agents": N_agents,
            "N_total_food_units": N_total_food_units,
            "edge_size": edge_size,
        }
    ]
).to_csv(os.path.join(home_dir, "additional_meta_params.csv"))

display(metadataDF)

print(metadataDF.shape)


,c_trust,sight_radius,reward_patch_dim,sim index
0,0.000,5,1,0
1,0.000,5,2,1
2,0.000,5,4,2
3,0.005,5,1,3
4,0.005,5,2,4
...,...,...,...,...
295,0.680,5,2,295
296,0.680,5,4,296
297,0.690,5,1,297
298,0.690,5,2,298


(300, 4)


In [4]:
# start simulations from scratch or from where you left off
fresh_start = False
if fresh_start:
    start = 0
else:
    resultsDF = pd.read_csv(os.path.join(home_dir, "resultsDF.csv"))
    start = resultsDF.iloc[-1]["sim index"].astype(int)  # start with the last existing batch

    logging.info(f"Starting from batch {start+1}.")

all_results = []

for si in range(start, N_sims):
    # 1. pull out parameters from row si in the metadata
    df_row = metadataDF.iloc[[si]]
    c_trust = df_row["c_trust"].iloc[0]
    sight_radius = df_row["sight_radius"].iloc[0]
    reward_patch_dim = df_row["reward_patch_dim"].iloc[0].astype(int)

    # arrays to save success measures for each run of this simulation
    mean_times_to_first_reward = np.zeros((N_runs))
    num_birds_failed = np.zeros((N_runs))

    logging.info(f"Starting simulation setting {si+1}/{N_sims}, about to run it {N_runs} times.")

    # Do multiple runs of the simulation and store the results in a results dataframe
    batch_results = []
    for ri in range(N_runs):
        # initialize environment
        env = environments.Environment(
            edge_size=edge_size,
            N_total_food_units=N_total_food_units,
            patch_dim=reward_patch_dim,
        )
        env.add_food_patches()

        # run simulation
        sim = simulation.SimulateCommunicators(env, N_frames, N_agents, c_trust=c_trust, sight_radius=sight_radius)
        sim.run()

        # Compute success measures
        time_to_first_allbirds = np.zeros(N_agents)
        for bird_id in range(1, N_agents + 1):  # compute time to first food for each bird
            singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF.bird == bird_id]
            time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(
                singlebirdDF, sim.all_rewardsDF, N_frames
            )
        mean_times_to_first_reward = np.mean(time_to_first_allbirds)  # take the average across birds
        num_birds_failed = np.sum(time_to_first_allbirds == N_frames)  # number of birds that failed to reach food

        # Save the simulation results in a folder named sim{si}_run{ri} in the home directory
        sim_folder = "sim" + str(si) + "_run" + str(ri)
        sim_dir = os.path.join(home_dir, sim_folder)
        if not os.path.isdir(sim_dir):
            os.makedirs(sim_dir)
        sim.all_birdsDF.to_csv(os.path.join(sim_dir, "birdlocsDF.csv"))
        sim.all_rewardsDF.to_csv(os.path.join(sim_dir, "rewardlocsDF.csv"))

        # Combine the metadata and the success measures for the results dataframe
        results_onesim = {
            "c_trust": c_trust,
            "sight_radius": sight_radius,
            "reward_patch_dim": reward_patch_dim,
            "sim index": si,
            "run index": ri,
            "time to first food": mean_times_to_first_reward,
            "num birds failed": num_birds_failed,
        }
        batch_results.append(results_onesim)

    batch_resultsDF = pd.DataFrame(batch_results)

    if "resultsDF" in locals():
        resultsDF = pd.concat([resultsDF, batch_resultsDF], ignore_index=True, axis=0)
    else:
        resultsDF = batch_resultsDF.copy()

    resultsDF.to_csv(os.path.join(home_dir, "resultsDF.csv"))
    logging.info(f"Saved results for batch {si+1}/{N_sims}.")

2023-09-20 21:42:46,074:  Starting from batch 58.
2023-09-20 21:42:46,079:  Starting simulation setting 58/300, about to run it 2 times.


2023-09-20 21:46:26,565:  Saved results for batch 58/300.
2023-09-20 21:46:26,566:  Starting simulation setting 59/300, about to run it 2 times.
2023-09-20 21:49:29,916:  Saved results for batch 59/300.
2023-09-20 21:49:29,917:  Starting simulation setting 60/300, about to run it 2 times.
2023-09-20 21:52:10,599:  Saved results for batch 60/300.
2023-09-20 21:52:10,601:  Starting simulation setting 61/300, about to run it 2 times.
2023-09-20 21:54:46,946:  Saved results for batch 61/300.
2023-09-20 21:54:46,948:  Starting simulation setting 62/300, about to run it 2 times.
2023-09-20 21:58:10,063:  Saved results for batch 62/300.
2023-09-20 21:58:10,064:  Starting simulation setting 63/300, about to run it 2 times.
2023-09-20 22:01:05,401:  Saved results for batch 63/300.
2023-09-20 22:01:05,402:  Starting simulation setting 64/300, about to run it 2 times.
2023-09-20 22:03:58,156:  Saved results for batch 64/300.
2023-09-20 22:03:58,157:  Starting simulation setting 65/300, about to r

KeyboardInterrupt: 

In [9]:
# animate a simulation
# load the data from the one simulation
sim_folder = "sim0_run0"
sim_dir = os.path.join(home_dir, sim_folder)
birdlocsDF = pd.read_csv(os.path.join(sim_dir, "birdlocsDF.csv"), index_col=0)
rewardlocsDF = pd.read_csv(os.path.join(sim_dir, "rewardlocsDF.csv"), index_col=0)

# animate the last simulation
communicators = ft.object_from_data(birdlocsDF, rewardlocsDF)
ft.animate_birds(communicators, plot_rewards=True, width=600, height=600, point_size=10)

See histograms.ipynb to generate histograms